In [245]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import random
from collections import Counter
from itertools import groupby
from concurrent.futures import ThreadPoolExecutor
import threading


In [227]:
flat_file = pd.read_csv('fotmob_odds_df.csv', index_col = [0])

In [228]:
def calculate_streaks(form):
    win_streak = 0
    loss_streak = 0
    winless_streak = 0
    
    for result in reversed(form):
        if result == 'W':
            win_streak += 1
            winless_streak = 0
            if loss_streak > 0:
                loss_streak = 0
        elif result == 'L':
            loss_streak += 1
            winless_streak += 1
            if win_streak > 0:
                win_streak = 0
        elif result == 'D':
            winless_streak += 1
            win_streak = 0
            loss_streak = 0
            if win_streak > 0:
                win_streak = 0
    
    return win_streak, loss_streak, winless_streak

def calculate_team_stats(matches, team):
    goals_scored = 0
    goals_conceded = 0
    wins = 0
    draws = 0
    losses = 0
    clean_sheets = 0
    failed_to_score = 0
    form = []

    for match in matches:
        is_home_team = match['home_team'] == team
        team_goals, opponent_goals = map(int, match['score'].split(' - '))
        if not is_home_team:
            team_goals, opponent_goals = opponent_goals, team_goals

        goals_scored += team_goals
        goals_conceded += opponent_goals

        if team_goals > opponent_goals:
            form.append('W')
            wins += 1
        elif team_goals < opponent_goals:
            form.append('L')
            losses += 1
        else:
            form.append('D')
            draws += 1

        if opponent_goals == 0:
            clean_sheets += 1

        if team_goals == 0:
            failed_to_score += 1

    win_streak, loss_streak, winless_streak = calculate_streaks(form)

    features = {
        'goals_scored': goals_scored,
        'goals_conceded': goals_conceded,
        'goal_difference': goals_scored - goals_conceded,
        'wins': wins,
        'draws': draws,
        'losses': losses,
        'points_gained': 3*wins + 1*draws + 0*losses,
        'win_ratio': wins / len(matches) if len(matches) else 0,
        'draw_ratio': draws / len(matches) if len(matches) else 0,
        'loss_ratio': losses / len(matches) if len(matches) else 0,
        'win_streak': win_streak,
        'loss_streak': loss_streak,
        'winless_streak': winless_streak,
        'average_goals_scored': goals_scored / len(matches) if len(matches) else 0,
        'average_goals_conceded': goals_conceded / len(matches) if len(matches) else 0,
        'clean_sheets': clean_sheets,
        'failed_to_score': failed_to_score,
        'scoring_ratio': (len(matches) - failed_to_score) / len(matches) if len(matches) else 0,
        'conceding_ratio': (len(matches) - clean_sheets) / len(matches) if len(matches) else 0,
    }

    return features

def summarize_team_performance(data):
    team_counter = Counter([match['home_team'] for match in data] + [match['away_team'] for match in data])
    teams = [team for team, freq in team_counter.most_common() if freq >= 5]

    home_team, away_team = teams[0], teams[1]

    home_matches, away_matches = data[:5], data[5:10]

    home_form = calculate_team_stats(home_matches, home_team)
    away_form = calculate_team_stats(away_matches, away_team)

    home_form = {'home_form_' + key: value for key, value in home_form.items()}
    away_form = {'away_form_' + key: value for key, value in away_form.items()}

    return {**home_form, **away_form}

In [229]:
def scrape_form(url):
#     url = flat_file['url'][3]
    page = requests.get(url)

    # pause for interval between 0 and 1 seconds to avoid getting banned
    time.sleep(random.randint(0, 2))

    soup = BeautifulSoup(page.content, 'html.parser')

    # find all 'ul' elements
    elements = soup.find_all('ul')

    matches = []
    for element in elements:
        # find all 'li' within the 'ul'
        li_elements = element.find_all('li')

        for li_element in li_elements:
            # find 'a' with the specific classes for right and left containers, and for the bottom elements
            a_element = li_element.find('a', {'class': ['right css-skyz2k-TeamFormContainer e3w5gu46',
                                                        'left css-skyz2k-TeamFormContainer e3w5gu46',
                                                        'right css-1ac4ee9-TeamFormContainer e3w5gu46',
                                                        'left css-1ac4ee9-TeamFormContainer e3w5gu46']})
            if a_element is not None:
                teams = a_element.find_all('span', class_='css-1lje8ql-TeamName e3w5gu40')
                score_div = a_element.find('div', {'class': ['css-la90e9-ResultBox ecz4wo12',
                                                             'css-udltjo-ResultBox ecz4wo12',
                                                             'css-1ef1lvo-ResultBox ecz4wo12']})
                if score_div is not None and len(teams) == 2:
                    score = score_div.span.text
                    match = {
                        'home_team': teams[0].text,
                        'away_team': teams[1].text,
                        'score': score,
                    }
                    matches.append(match)
    matches = matches[:10]
    return matches

In [230]:
def process_url(url):
    matches = scrape_form(url)
    stats = summarize_team_performance(matches)
    return {'url': url, 'stats': stats}

def chunked_scrape(urls, chunk_size=50, sleep_interval=2):
    num_urls = len(urls)
    stats_list = []

    for i in range(0, num_urls, chunk_size):
        print(f"processing {i} to {i + chunk_size}...")
        chunk_urls = urls[i:i+chunk_size]

        with ThreadPoolExecutor(max_workers=5) as executor:
            chunk_stats = list(executor.map(process_url, chunk_urls))

        stats_list.extend(chunk_stats)
        if i + chunk_size < num_urls:
            time.sleep(sleep_interval)

    return stats_list

urls = flat_file['url'].tolist()

stats_list = chunked_scrape(urls)

url_list = [stat['url'] for stat in stats_list]
stats_list = [stat['stats'] for stat in stats_list]

stats_df = pd.DataFrame(stats_list)
stats_df['url'] = url_list

Processing 0 to 50...
Processing 50 to 100...
Processing 100 to 150...
Processing 150 to 200...
Processing 200 to 250...
Processing 250 to 300...
Processing 300 to 350...
Processing 350 to 400...
Processing 400 to 450...
Processing 450 to 500...
Processing 500 to 550...
Processing 550 to 600...
Processing 600 to 650...
Processing 650 to 700...
Processing 700 to 750...
Processing 750 to 800...
Processing 800 to 850...
Processing 850 to 900...
Processing 900 to 950...
Processing 950 to 1000...
Processing 1000 to 1050...
Processing 1050 to 1100...
Processing 1100 to 1150...
Processing 1150 to 1200...
Processing 1200 to 1250...
Processing 1250 to 1300...
Processing 1300 to 1350...
Processing 1350 to 1400...
Processing 1400 to 1450...
Processing 1450 to 1500...
Processing 1500 to 1550...
Processing 1550 to 1600...
Processing 1600 to 1650...
Processing 1650 to 1700...
Processing 1700 to 1750...
Processing 1750 to 1800...
Processing 1800 to 1850...
Processing 1850 to 1900...
Processing 1900 t

In [305]:
combined_df = pd.concat([flat_file.reset_index(drop=True), stats_df.reset_index(drop=True)], axis=1)

In [252]:
def scrape_url(url):
    page = requests.get(url)
    time.sleep(random.randint(0, 1))
    soup = BeautifulSoup(page.content, 'html.parser')
    league_name = soup.find('span').text
    return {'url': url, 'league_name': league_name}

def scrape_league_names(urls):
    results = []

    chunk_size = 100
    num_chunks = len(urls) // chunk_size + (len(urls) % chunk_size > 0)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i in range(num_chunks):
            urls_chunk = urls[i * chunk_size: (i + 1) * chunk_size]
            for url in urls_chunk:
                future = executor.submit(scrape_url, url)
                futures.append(future)
            concurrent.futures.wait(futures)
            print(f"processed {len(urls_chunk)} links {i + 1}/{num_chunks}")

        for future in futures:
            results.append(future.result())

    df = pd.DataFrame(results)
    return df


In [253]:
urls = flat_file['url'].tolist()
league_name_df = scrape_league_names(urls)

Processed 100 links 1/27
Processed 100 links 2/27
Processed 100 links 3/27
Processed 100 links 4/27
Processed 100 links 5/27
Processed 100 links 6/27
Processed 100 links 7/27
Processed 100 links 8/27
Processed 100 links 9/27
Processed 100 links 10/27
Processed 100 links 11/27
Processed 100 links 12/27
Processed 100 links 13/27
Processed 100 links 14/27
Processed 100 links 15/27
Processed 100 links 16/27
Processed 100 links 17/27
Processed 100 links 18/27
Processed 100 links 19/27
Processed 100 links 20/27
Processed 100 links 21/27
Processed 100 links 22/27
Processed 100 links 23/27
Processed 100 links 24/27
Processed 100 links 25/27
Processed 100 links 26/27
Processed 33 links 27/27


In [288]:
def extract_league_name(data_list):
    processed_list = []
    for item in data_list:
        if ' Grp' in item:
            item = item.split(' Grp.')[0]
        elif ' Round' in item:
            item = item.split(' Round')[0]
        elif ' Quarter' in item:
            item = item.split(' Quarter')[0]
        processed_list.append(item)
    return processed_list

In [306]:
league_names = extract_league_name(league_name_df['league_name'])

In [307]:
combined_df['league_name'] = league_names

In [308]:
combined_df = combined_df.iloc[:, :-1]


In [309]:
combined_df

,expected_goals_(xg)_diff,total_shots_diff,big_chances_diff,big_chances_missed_diff,accurate_passes_diff,accurate_passes_percentage_diff,fouls_committed_diff,offsides_diff,corners_diff,shots_off_target_diff,shots_on_target_diff,blocked_shots_diff,hit_woodwork_diff,shots_inside_box_diff,shots_outside_box_diff,xg_open_play_diff,xg_set_play_diff,xg_on_target_(xgot)_diff,passes_diff,own_half_diff,opposition_half_diff,accurate_long_balls_diff,accurate_long_balls_percentage_diff,accurate_crosses_diff,accurate_crosses_percentage_diff,throws_diff,yellow_cards_diff,red_cards_diff,tackles_won_diff,tackles_won_percentage_diff,interceptions_diff,blocks_diff,clearances_diff,keeper_saves_diff,duels_won_diff,ground_duels_won_diff,ground_duels_won_percentage_diff,aerial_duels_won_diff,aerial_duels_won_percentage_diff,successful_dribbles_diff,successful_dribbles_percentage_diff,score_diff,posession_diff,url,odds_url,odds_sum,xg_penalty_diff,league_name,odds_predict,std_0,std_1,std_2,median_0,median_1,median_2,75_0,75_1,75_2,skew_0,skew_1,skew_2,kurtosis_0,kurtosis_1,kurtosis_2,range_0,range_1,range_2,mean_0,mean_1,mean_2,cv_odds,target,home_form_goals_scored,home_form_goals_conceded,home_form_goal_difference,home_form_wins,home_form_draws,home_form_losses,home_form_points_gained,home_form_win_ratio,home_form_draw_ratio,home_form_loss_ratio,home_form_win_streak,home_form_loss_streak,home_form_winless_streak,home_form_average_goals_scored,home_form_average_goals_conceded,home_form_clean_sheets,home_form_failed_to_score,home_form_scoring_ratio,home_form_conceding_ratio,away_form_goals_scored,away_form_goals_conceded,away_form_goal_difference,away_form_wins,away_form_draws,away_form_losses,away_form_points_gained,away_form_win_ratio,away_form_draw_ratio,away_form_loss_ratio,away_form_win_streak,away_form_loss_streak,away_form_winless_streak,away_form_average_goals_scored,away_form_average_goals_conceded,away_form_clean_sheets,away_form_failed_to_score,away_form_scoring_ratio,away_form_conceding_ratio
0,0.52,-1.0,2.0,1.0,-21.0,0.00,-3.0,0.0,1.0,1.0,-2.0,0.0,0.0,-1.0,0.0,0.37,0.14,0.37,-23.0,-34.0,13.0,13.0,0.31,4.0,0.29,3.0,0.0,0.0,-4.0,-0.50,-2.0,0.0,-11.0,3.0,10.0,6.0,0.20,4.0,0.28,3.0,0.17,1.0,-2.0,https://www.fotmob.com/match/3887480/matchfact...,https://www.oddschecker.com/football/denmark/s...,1.083333,0.00,Superligaen,1,0.210499,0.020865,1.051856,4.5000,1.285714,12.000000,4.7500,1.285714,13.000000,0.636688,0.122564,0.598560,-0.452777,-1.140474,-0.511646,0.766667,0.065789,3.750000,4.628699,1.274710,12.014706,0.927398,1,7,7,0,2,1,2,7,0.4,0.2,0.4,0,1,2,1.4,1.4,2,1,0.8,0.6,4,6,-2,1,1,3,4,0.2,0.2,0.6,0,1,2,0.8,1.2,1,2,0.6,0.8
1,-0.52,-4.0,0.0,0.0,-41.0,-0.05,3.0,2.0,1.0,-3.0,1.0,-2.0,0.0,-3.0,-1.0,-0.53,0.01,1.21,-34.0,29.0,-70.0,5.0,-0.10,0.0,0.03,-8.0,0.0,0.0,1.0,0.07,1.0,3.0,6.0,-2.0,4.0,1.0,0.04,3.0,0.16,3.0,0.19,0.0,-8.0,https://www.fotmob.com/match/3900377/matchfact...,https://www.oddschecker.com/football/netherlan...,1.073680,0.00,Eredivisie,2,0.062549,0.104473,0.061710,2.7000,3.600000,2.350000,2.7500,3.623529,2.400000,0.319652,0.170352,-0.167191,-0.576020,-0.746084,-1.225969,0.221154,0.350000,0.200000,2.710324,3.560372,2.340443,0.223648,2,7,15,-8,0,1,4,1,0.0,0.2,0.8,0,4,5,1.4,3.0,0,1,0.8,1.0,6,9,-3,0,3,2,3,0.0,0.6,0.4,0,2,5,1.2,1.8,1,2,0.6,0.8
2,0.40,6.0,0.0,0.0,231.0,0.22,1.0,0.0,8.0,2.0,0.0,4.0,1.0,6.0,0.0,0.18,0.22,-0.20,230.0,91.0,140.0,9.0,0.32,6.0,0.26,2.0,-1.0,0.0,-2.0,-0.41,1.0,-4.0,-14.0,-1.0,14.0,5.0,0.14,9.0,0.60,3.0,0.16,-1.0,46.0,https://www.fotmob.com/match/3937426/matchfact...,https://www.oddschecker.com/football/portugal/...,1.062790,0.00,Liga Portugal,1,0.108311,0.050475,0.220636,3.2000,1.952381,4.200000,3.2500,2.000000,4.250000,-0.489228,-0.946235,-0.439525,-1.303386,0.023974,-1.229584,0.300000,0.166667,0.686275,3.160526,1.953566,4.063142,0.361217,1,12,2,10,5,0,0,15,1.0,0.0,0.0,5,0,0,2.4,0.4,4,0,1.0,0.2,5,8,-3,1,2,2,5,0.2,0.4,0.4,0,0,3,1.0,1.6,1,1,0.8,0.8
3,0.00,8.0,3.0,2.0,34.0,0.01,0.0,-1.0,1.0,5.0,3.0,0.0,0.0,7.0,1.0,

In [310]:
combined_df.to_csv('fotmob_odds_form_df.csv')